In [84]:
import numpy as np

# Gerando a matriz A e o vetor b
Seja a matriz A e o vetor b das formas:

$$A = 
\begin{bmatrix}
9 & -4 & 1 & 0 & \cdots & \cdots & 0\\
-4 & 6 & -4 & 1 & \ddots & & \vdots\\
1 & -4 & 6 & -4 & 1 & \ddots & \vdots\\
0 & \ddots & \ddots & \ddots & \ddots & \ddots & 0\\
\vdots & \ddots & 1 & -4 & 6 & -4 & 1\\
\vdots & & \ddots & 1 & -4 & 5 & -2\\
0 & \cdots & \cdots & 0 & 1 & -2 & 1\\
\end{bmatrix}, b = 
\begin{bmatrix}
n^{-4}\\
n^{-4}\\
\vdots\\
\vdots\\
\vdots\\
n^{-4}\\
n^{-4}\\
\end{bmatrix}
$$

In [95]:
# cria uma matriz A e um vetor b da forma pre-definida
def generate_matrix(n=100):
    A = np.zeros((n, n)) # cria matrix A nxn

    # preenche as 5 diagonais
    np.fill_diagonal(A, 6)
    np.fill_diagonal(A[:-1, 1:], -4)
    np.fill_diagonal(A[1:, :-1], -4)
    np.fill_diagonal(A[:-2, 2:], 1)
    np.fill_diagonal(A[2:, :-2], 1)

    # preenche valores diferentes
    A[0, 0] = 9
    A[-1, -1] = 1
    A[-2, -2] = 5
    A[-2, -1] = -2
    A[-1, -2] = -2
    
    # cria o vetor b
    b = np.full((n, 1), n**-4)
    
    return A, b

# Decomposição de Cholesky

Uma matriz $A$ simétrica e definida positiva pode ser decomposta na forma:
$$A = GG^T$$

Em que $G$ é uma matriz triangular inferior.

## A decomposição

### Elementos diagonais

$$\begin{cases}
g_{11} = \sqrt{a_{11}} \\
g_{ii} = \sqrt{a_{ii} - \sum_{k=1}^{i-1}g_{ik}^2}, i = 2, 3, \cdots, n\\
\end{cases} $$

### Elementos não diagonais

$$\begin{cases}
g_{i1} = \frac{a_{i1}}{g_{11}}, i = 2, 3, \cdots, n\\
g_{ij} = \frac{a_{ij} - \sum_{k=1}^{j-1}g_{ik}g_{jk}}{g_{jj}}, 2 < j < i
\end{cases} $$

In [88]:
from math import sqrt

In [89]:
def cholesky_decomposition(A):
    G = np.zeros_like(A)
    n = A.shape[0]
    
    G[0,0] = sqrt(A[0,0])
    for i in range(1, n):
        G[i,0] = A[i,0] / G[0,0]
        for j in range(1, i):
            sum = 0
            for k in range(j):
                sum += G[i,k]*G[j,k]
            G[i,j] = A[i,j] - sum
            G[i,j] /= G[j,j]
        sum = 0
        for k in range(i):
            sum += G[i,k]**2
        G[i,i] = sqrt(A[i,i] - sum)
    return G

In [101]:
# verifica se uma matriz é simétrica, com um erro definido
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

# verifica se a matriz é simétrica e definida positiva
def check_cholesky(A):
    return check_symmetric(A) and np.all(np.linalg.eigvals(A) > 0)

In [100]:
A = np.array([[3, 4, 3], [4, 8, 6], [3, 6, 9]]).astype('float64')
print(cholesky_decomposition(A))

[[1.73205081 0.         0.        ]
 [2.30940108 1.63299316 0.        ]
 [1.73205081 1.22474487 2.12132034]]


## Resolução de Sistemas

Para resolver um sistema do tipo $Ax = b$ com decomposição de Cholesky, pode-se fazer como na decomposição L.U:

$$Ax = b$$
$$(GG^T)x = b$$
$$Gy = b$$
$$G^Tx = y$$

De modo a simplificar as contas, afinal $G$ é uma matriz triangular inferior e $G^T$ uma matriz triangular superior.

In [106]:
# resolve A * x = b, em que A é uma matriz triangular inferior
def lower_solve(A, b):
    n = A.shape[0]
    for k in range(1, n):
        b[k] -= A[k, 0:k].dot(b[0:k])
    return b

# resolve A * x = b, em que A é uma matriz triangular superior
def upper_solve(A, b):
    n = A.shape[0]
    for k in range(n-1, -1, -1):
        b[k] -= A[k, k+1:n].dot(b[k+1:n])
        b[k] /= A[k,k]
    return b

# resolve Ax = b por decomposição de cholesky
def cholesky_solve(A, b):
    G1 = cholesky_decomposition(A)
    G2 = G1.T
    
    # divide em duas resoluções de sistema
    y = lower_solve(G1, b)
    x = upper_solve(G2, y)
    
    # retorna o resultado
    return x

In [107]:
A, b = generate_matrix()
x = cholesky_solve(A, b)

# Referências

Decomposição de Cholesky: http://wwwp.fc.unesp.br/~arbalbo/Iniciacao_Cientifica/sistemaslineares/teoria/3_Met_Cholesky.pdf

Teste de matriz simetrica em python: https://stackoverflow.com/questions/42908334/checking-if-a-matrix-is-symmetric-in-numpy